# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 -- Create Employees table
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Department VARCHAR(50),
    HireDate DATE
);

-- Insert sample data into Employees
INSERT INTO Employees (EmployeeID, FirstName, LastName, Department, HireDate)
VALUES
    (1, 'John', 'Doe', 'Sales', '2020-01-15'),
    (2, 'Jane', 'Smith', 'Marketing', '2019-05-20'),
    (3, 'Mike', 'Johnson', 'IT', '2021-03-10'),
    (4, 'Emily', 'Brown', 'HR', '2018-11-01'),
    (5, 'David', 'Lee', 'Finance', '2022-07-05');

-- Create Projects table
CREATE TABLE Projects (
    ProjectID INT PRIMARY KEY,
    ProjectName VARCHAR(100),
    StartDate DATE,
    EndDate DATE,
    Budget DECIMAL(10, 2)
);

-- Insert sample data into Projects
INSERT INTO Projects (ProjectID, ProjectName, StartDate, EndDate, Budget)
VALUES
    (101, 'Website Redesign', '2023-01-01', '2023-06-30', 50000.00),
    (102, 'Mobile App Development', '2023-03-15', '2023-12-31', 75000.00),
    (103, 'Data Migration', '2023-05-01', '2023-08-31', 30000.00),
    (104, 'AI Integration', '2023-07-01', '2024-02-29', 100000.00);

-- Create EmployeeProjects table (junction table for many-to-many relationship)
CREATE TABLE EmployeeProjects (
    EmployeeID INT,
    ProjectID INT,
    Role VARCHAR(50),
    PRIMARY KEY (EmployeeID, ProjectID),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID),
    FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID)
);

-- Insert sample data into EmployeeProjects
INSERT INTO EmployeeProjects (EmployeeID, ProjectID, Role)
VALUES
    (1, 101, 'Project Manager'),
    (2, 101, 'Designer'),
    (3, 102, 'Lead Developer'),
    (4, 103, 'Data Analyst'),
    (5, 104, 'Financial Advisor'),
    (1, 102, 'Sales Representative'),
    (3, 104, 'AI Specialist');
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
    -- Query 1: List all employees and their departments
    -- Query 2: Show all projects with their total budget
    -- Query 3: Find employees working on multiple projects
    -- Query 4: List projects and the number of employees working on each
    -- Query 5: Find the most expensive project and its project manager
"""
})

In [5]:
#Function to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Natural Language: Who are all the employees in the IT department?""", context_user))

```sql
SELECT employee_name
FROM employees
WHERE department = 'IT';
```


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which employee is working on the most projects?", old_context_user))

This is your SQL:
```sql
SELECT e.name, COUNT(p.ID_project) AS project_count
FROM employees e
JOIN projects p ON e.ID_usr = p.ID_usr
GROUP BY e.name
ORDER BY project_count DESC
LIMIT 1;
```

This SQL query selects the employee's name and counts the number of projects they are working on by joining the employees table with the projects table on the employee's ID. It then groups the results by employee name, orders them by the project count in descending order, and limits the output to only show the employee with the most projects.


In [9]:
#new
print(return_CCRMSQL("List all employees hired in 2020 or later.", context_user))

```sql
SELECT *
FROM employees
WHERE hire_date >= '2020-01-01';
```


In [10]:
#old
print(return_CCRMSQL("What are the names and roles of employees working on the AI Integration project?", old_context_user))

This is your SQL:
```sql
SELECT employees.name, studies.Speciality
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.Speciality = 'AI Integration';
```

This SQL query selects the names of employees and their roles who are working on the AI Integration project by joining the "employees" and "studies" tables on the ID_usr field and filtering the results based on the Speciality column being 'AI Integration'.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?